In [157]:
import cv2
import numpy as np

# โหลดรูปภาพ
#image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/high_school/transcript_hs_front_3.png")
#image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/transcript_hs_front_1.png")
image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/transcript_hs_front_2.png")

#image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/pun_tran_mo.jpg")
#image = cv2.imread("E:/document.jpg")
denoised = cv2.bilateralFilter(image, 20, 30, 30)
gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edged = cv2.Canny(gray_img, 50, 200)

# สำหรับภาพที่แสงไม่สม่ำเสมอ
binary_gaussian = cv2.adaptiveThreshold(
    gray_img, 
    maxValue=255, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=41, 
    C=20
)

cv2.imwrite("output/transcript_binary_g.png", binary_gaussian)
cv2.imwrite("output/gray_img.png", gray_img)
cv2.imwrite("output/edged.png", edged)


True

In [158]:
original_img = image
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_gaussian, connectivity=8)
areas = [stat[4] for stat in stats]  # ดึงค่า area
sorted_areas = sorted(areas, reverse=True)  # เรียงลำดับจากมากไปน้อย
second_max_area = sorted_areas[1]  # ค่าอันดับ 2
second_max_area_index = areas.index(second_max_area)  # หาตำแหน่งในลิสต์เดิม

table_position = stats[second_max_area_index]
x, y, w, h, area = table_position
table_img = binary_gaussian[y:y+h, x:x+w]
table_original = original_img[y:y+h, x:x+w]
cv2.imwrite("output/table.png", table_img)
cv2.imwrite("output/table_original.png", table_original)


True

In [159]:
def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        #print(area)
        if area > 1000:
            #print("มา")
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area

    return biggest

image_copy = image.copy()
table_original_copy = table_original.copy()

contours, hierarchy = cv2.findContours(table_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

biggest = biggest_contour(contours)
#for i in contours:
#    cv2.drawContours (image_copy, [i], -1, (0, 255, 0), 3)

#cv2.drawContours(image_copy, [biggest], -1, (0, 255, 0), 3)
#cv2.imwrite("output/contours.png", image_copy)

print(biggest)

points = biggest.reshape(4, 2)
input_points = np.zeros((4, 2), dtype="float32")

points_sum = points.sum(axis=1)
input_points[0] = points[np.argmin(points_sum)]
input_points[3] = points[np.argmax(points_sum)]

points_diff = np.diff(points, axis=1)
input_points[1] = points[np.argmin(points_diff)]
input_points[2] = points[np.argmax(points_diff)]

(top_left, top_right, bottom_right, bottom_left) = input_points

# Euclidean Distance Formula
bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
rigth_height = np.sqrt(((top_left[0] - bottom_right[0]) ** 2) + ((top_left[1] - bottom_right[1]) ** 2))
left_height = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

# Output image size
#max_width = max(int(bottom_width), int(top_width))
expand_width = round(max(int(bottom_width), int(top_width)) * 0.4)
max_width = max(int(bottom_width), int(top_width)) + expand_width
max_height = max(int(rigth_height), int(left_height))

# Desird points values in the output image
converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])

# Perspective transformaxtion
matrix = cv2.getPerspectiveTransform(input_points, converted_points)
img_out = cv2.warpPerspective(table_original_copy, matrix, (max_width, max_height))
#img_out = cv2.warpPerspective(image_copy, matrix, (max_width, max_height))



cv2.imwrite("output/warp_perspective.png", img_out)
#cv2.imwrite("output/contours.png", table_original_copy)

[[[1539    0]]

 [[  15   39]]

 [[   0 1318]]

 [[1571 1348]]]


True

In [145]:
import cv2
import numpy as np
from skimage.transform import PiecewiseAffineTransform, warp

def biggest_contour(contours):
    """ ค้นหาคอนทัวร์ที่ใหญ่ที่สุดที่มี 4 มุม (สี่เหลี่ยม) """
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 1000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest

# โหลดภาพ
image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/transcript_hs_front_2.png")
image_copy = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ค้นหาขอบของภาพ
edges = cv2.Canny(gray, 50, 150)

# ค้นหาคอนทัวร์
contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# ค้นหาสี่เหลี่ยมที่ใหญ่ที่สุด
biggest = biggest_contour(contours)

# ตรวจสอบว่าพบตารางหรือไม่
if biggest.size == 0:
    print("ไม่พบสี่เหลี่ยมในภาพ")
    exit()

# แปลงรูปแบบของจุดมุม
points = biggest.reshape(4, 2)

# สร้างจุดควบคุมเพิ่มเติม (4 มุม + จุดกลางของแต่ละด้าน)
extra_points = [
    (points[0] + points[1]) / 2,  # กลางบน
    (points[1] + points[2]) / 2,  # ขวากลาง
    (points[2] + points[3]) / 2,  # ล่างกลาง
    (points[3] + points[0]) / 2   # ซ้ายกลาง
]

# รวมจุดทั้งหมด (4 มุม + จุดกลาง)
src_points = np.vstack([points] + [np.array(extra_points)])

# คำนวณขนาดภาพปลายทาง
max_width = int(max(np.linalg.norm(points[1] - points[0]), np.linalg.norm(points[2] - points[3])))
max_height = int(max(np.linalg.norm(points[0] - points[3]), np.linalg.norm(points[1] - points[2])))

# กำหนดตำแหน่งจุดปลายทางที่ต้องการให้เป็นเส้นตรง
dst_points = np.array([
    [0, 0], [max_width, 0], [max_width, max_height], [0, max_height],  # 4 มุม
    [max_width // 2, 0], [max_width, max_height // 2],  # จุดกลางบน & ขวา
    [max_width // 2, max_height], [0, max_height // 2]  # จุดกลางล่าง & ซ้าย
], dtype=np.float32)

# ใช้ Thin Plate Spline (TPS) - Piecewise Affine Transform
tps = PiecewiseAffineTransform()
tps.estimate(dst_points, src_points)

# ทำการแปลงภาพ
warped = warp(image, tps, output_shape=(max_height, max_width))

# แปลงกลับเป็น uint8 เพื่อใช้กับ OpenCV
warped = (warped * 255).astype(np.uint8)

# แสดงผลภาพที่ถูกแก้ไข
cv2.imwrite("output/TPS.png", warped)


True